In [1]:
import pandas as pd
from docx import Document
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from docx.shared import RGBColor
from docx.shared import Cm
from docx.enum.table import WD_ROW_HEIGHT_RULE
import math
import re

In [2]:
def create_page(document):
    
    table = document.add_table(32,4)
    table.style = 'Table Grid'
    
    return table

In [3]:
def merge_cells(table, row, max_cell, start_cell=0):
    
    merge_anchor = table.cell(row, start_cell)
    for i in range(start_cell+1, max_cell+1):
        merge_anchor.merge(table.cell(row, i))
    
    return merge_anchor
        
    

In [4]:
def blue_cell(cell):
    
    # Set a cell background (shading) color to RGB D9D9D9.
    shading_elm = parse_xml(r'<w:shd {} w:fill="4682B4"/>'.format(nsdecls('w')))
    cell._tc.get_or_add_tcPr().append(shading_elm)

In [5]:
def write_white(cell, text):
    
    text = cell.paragraphs[0].add_run(text)
    white = RGBColor(255,255,255)
    text.font.color.rgb = white

In [6]:
def white_on_blue(cell, text):
    
    blue_cell(cell)
    write_white(cell, text)

In [7]:
def white_row(table, row_num):
    
    merge_cells(table, row_num, max_cell=3)
    row = table.rows[row_num]
    row.height_rule = WD_ROW_HEIGHT_RULE.EXACTLY
    row.height = Cm(.3)


In [8]:
def add_header(table, num_linha):
    
    header_text = 'Formulário de Ações - Plano de Retomada Econômica pós-pandemia'
    merged_header = merge_cells(table, 0, 3)
    white_on_blue(merged_header, header_text)
    num_linha_cell = merge_cells(table, 1, 3)
    num_linha_cell.text = str(num_linha)

In [9]:
def add_secretaria(table, data_row):
    
    key_secretaria = merge_cells(table, 2, 1)
    white_on_blue(key_secretaria, 'Secretaria:')
    
    val_secretaria = merge_cells(table, 2, 3, start_cell=2)
    val_secretaria.text = data_row['secretaria']

In [10]:
def add_projeto(table, data_row):
    
    key_projeto = merge_cells(table, 3, 1)
    white_on_blue(key_projeto, 'Programa/Projeto:')
    
    val_projeto = merge_cells(table, 3, 3, start_cell=2)
    val_projeto.text = data_row['projeto']
    white_row(table, 4)

In [11]:
def demais_orgaos(table, data_row):
    
    header_demais_orgaos = merge_cells(table, 5, 3)
    white_on_blue(header_demais_orgaos,'Demais órgãos participantes/interessados')
    val_demais_orgaos = merge_cells(table, 6, 3)
    val_demais_orgaos.text = data_row['demais_orgaos_participantes']
    white_row(table, 7)

In [12]:
def estagio_atual_projeto(table, data_row):
    
    header_estagio = merge_cells(table, 8, 3)
    white_on_blue(header_estagio,"Estágio atual do Projeto")
    table.cell(9, 0).text = "Em andamento"
    table.cell(9, 1).text=data_row['andamento']
    table.cell(9, 2).text="Ação futura"
    table.cell(9,3).text=data_row['acao_futura']
    desc_estagio_cell = merge_cells(table, 10, 3)
    desc_estagio_cell.text=data_row['estagio_atual']
    white_row(table, 11)

In [13]:
def orcamento_proj(table, data_row):
    
    header_orcamento = merge_cells(table, 12, 3)
    header_orcamento.text = "Valor"
    white_on_blue(table.cell(13, 0), 'Ano')
    white_on_blue(table.cell(13, 1), 'Investimento (R$ mil)')
    white_on_blue(table.cell(13, 2), 'Custerio (R$ mil)')
    white_on_blue(table.cell(13, 3), 'Informações Complementares')
    
    table.cell(14, 0).text = data_row['desc_ano_1']
    table.cell(14, 1).text = data_row['investimento_ano_1']
    table.cell(14, 2).text = data_row['custeio_ano_1']
    table.cell(14, 3).text = data_row['info_complementares_ano_1']
    table.cell(15, 0).text = data_row['desc_ano_2']
    table.cell(15, 1).text = data_row['investimento_ano_2']
    table.cell(15, 2).text = data_row['custeio_ano_2']
    table.cell(15, 3).text = data_row['info_complementares_ano_2']
    table.cell(16, 0).text = data_row['desc_ano_3']
    table.cell(16, 1).text = data_row['investimento_ano_3']
    table.cell(16, 2).text = data_row['custeio_ano_3']
    table.cell(16, 3).text = data_row['info_complementares_ano_3']
    table.cell(17, 0).text = data_row['desc_ano_4']
    table.cell(17, 1).text = data_row['investimento_ano_4']
    table.cell(17, 2).text = data_row['custeio_ano_4']
    table.cell(17, 3).text = data_row['info_complementares_ano_4']
    
    white_row(table, 18)

In [14]:
def inicio_e_termino_previsto(table, data_row):
    
    white_on_blue(table.cell(19, 0), 'Início previsto:')
    white_on_blue(table.cell(19, 2), 'Término previsto:')
    table.cell(19, 1).text = data_row['inicio']
    table.cell(19, 3).text = data_row['termino']
    
    white_row(table, 20)

In [15]:
def objetivo_projeto(table, data_row):
    
    header_objetivo = merge_cells(table, 21, 3)
    white_on_blue(header_objetivo,"Objetivo do Programa ou Projeto")
    
    val_objetivo = merge_cells(table, 22, 3)
    val_objetivo.text = data_row['objetivo']
    
    white_row(table, 23)

In [16]:
def beneficiarios(table, data_row):
    
    header = merge_cells(table, 24, 3)
    white_on_blue(header,"Beneficiários:")
    
    val = merge_cells(table, 25, 3)
    val.text = data_row['beneficiarios']
    
    white_row(table, 26)

In [17]:
def empregos(table, data_row):
    
    header = merge_cells(table, 27, 3)
    white_on_blue(header,"Empregos gerados:")
    
    val = merge_cells(table, 28, 3)
    val.text = data_row['empregos_geados']
    
    white_row(table, 29)

In [18]:
def resultados_esperados(table, data_row):
    
    header = merge_cells(table, 30, 3)
    white_on_blue(header,"Resultados Esperados e Impactos:")
    
    val = merge_cells(table, 31, 3)
    val.text = data_row['resultados_e_impactos']

In [19]:
def write_project(document, data_row, num_linha):
    
    table = create_page(document)
    
    add_header(table, num_linha)
    add_secretaria(table, data_row)
    add_projeto(table, data_row)
    demais_orgaos(table, data_row)
    estagio_atual_projeto(table, data_row)
    orcamento_proj(table, data_row)
    inicio_e_termino_previsto(table, data_row)
    objetivo_projeto(table, data_row)
    beneficiarios(table, data_row)
    empregos(table, data_row)
    resultados_esperados(table, data_row)
    
    document.add_page_break()

In [20]:
def format_as_reais(item):
    
    if item.startswith('R$'):
        return item
    else:
        try:
            item = float(item)
            fracao, inteiro = math.modf(item)
            fracao = round(fracao, 2)
            fracao = str(fracao).split('.')[-1]
            if len(fracao)<2:
                fracao = fracao+'0'
            digitos = str(int(inteiro))
            digitos_com_ponto = []
            for i, num in enumerate(digitos[::-1]):
                if i>0 and i%3 ==0:
                    digitos_com_ponto.append('.')
                digitos_com_ponto.append(num)
            digitos_com_ponto.reverse()
            digitos = ''.join(digitos_com_ponto)
                    
            return f"R${digitos},{fracao}"
        except ValueError:
            return item

In [21]:
def format_as_data(data):
    
    if data:
        match = re.match('^\d{4}-\d{2}-\d{2}', data)
        if match:
            match = match.group()
            ano, mes, dia = match.split('-')
            return f'{dia}/{mes}/{ano}'
        else:
            return data
    else:
        return data

In [22]:
data = pd.read_excel('dados_extraidos_v2.xlsx')

In [23]:
data.keys()

Index(['Unnamed: 0', 'secretaria', 'projeto', 'demais_orgaos_participantes',
       'andamento', 'acao_futura', 'estagio_atual', 'inicio', 'termino',
       'objetivo', 'beneficiarios', 'empregos_geados', 'resultados_e_impactos',
       'desc_ano_1', 'investimento_ano_1', 'custeio_ano_1',
       'info_complementares_ano_1', 'desc_ano_2', 'investimento_ano_2',
       'custeio_ano_2', 'info_complementares_ano_2', 'desc_ano_3',
       'investimento_ano_3', 'custeio_ano_3', 'info_complementares_ano_3',
       'desc_ano_4', 'investimento_ano_4', 'custeio_ano_4',
       'info_complementares_ano_4'],
      dtype='object')

In [24]:
data.drop('Unnamed: 0', axis = 1, inplace = True)
data.fillna('', inplace=True)

In [25]:
for col in data.columns:
    data[col] = data[col].apply(str)

In [26]:
colunas_dinheiro = [
    'investimento_ano_1', 'custeio_ano_1', 
    'investimento_ano_2','custeio_ano_2', 
    'investimento_ano_3', 'custeio_ano_3', 
    'investimento_ano_4', 'custeio_ano_4'
]

for col in colunas_dinheiro:
    data[col] = data[col].apply(format_as_reais)

In [27]:
data['termino'] = data['termino'].apply(format_as_data)
data['inicio'] = data['inicio'].apply(format_as_data)

In [28]:
document = Document()

In [29]:
for i, row in data.iterrows():
    print(i)
    write_project(document, row, i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142


In [30]:
document.save('teste2.docx')